In [2]:
import numpy as np
import h5py
import cv2
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from mpl_toolkits.mplot3d import Axes3D
# supress color warning when making movie
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
#%matplotlib notebook
import time
%load_ext autoreload
import os
%autoreload 2
import sys
sys.path.append('./lib/')

from ipywidgets import Video, Image
from IPython.display import display
import numpy as np
import cv2
import base64

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets


from DataManagement import SplitdataManager, Filepaths

In [3]:
# the path to the experiment
experiment_path = '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/'  

In [4]:
# make a splitdata manager
splitman = SplitdataManager(experiment_path)

In [5]:
# gather the names of the splitdata folders with corresponding indices
splitdata_names = [triplet[0].split('/')[-1] for triplet in splitman.splitdata_paths]
splitdata_idxs = [i for i in range(len(splitdata_names))]
splitdata_name_to_idx_dict = dict(zip(splitdata_names, splitdata_idxs))
splitdata_name_to_paths = dict(zip(splitdata_names, splitman.splitdata_paths))


In [7]:
#  ---- choose a splitdata folder ----- #

# make a dropdown menue to choose which splitdata you want
splitdata_chooser = widgets.Dropdown(options=splitdata_name_to_idx_dict, value=0)

# create and initialize widget to hold the splitdata_idx
splitdata_idx = widgets.Output()
with splitdata_idx:
    display(0)
    
# create and initialize widget to hold the filepaths of the movies
splitdata_paths_widget_list = [widgets.Output() for _ in range(3)]
for i in range(3):
    splitdata_paths = splitdata_paths_widget_list[i]
    with splitdata_paths:
        display(splitman.splitdata_paths[0][i])

# create functions to update output widgets on changes to splitdata_chooser
def splitdata_chooser_eventhandler_splitdata_idx(change):
    splitdata_idx.clear_output()
    with splitdata_idx:
        display(change.new)
splitdata_chooser.observe(splitdata_chooser_eventhandler_splitdata_idx, names='value')
        
def splitdata_chooser_eventhandler_splitdata_paths(change):
    for i in range(3):
        splitdata_paths = splitdata_paths_widget_list[i]
        splitdata_paths.clear_output()
        with splitdata_paths:
            display(splitman.splitdata_paths[change.new][i])
splitdata_chooser.observe(splitdata_chooser_eventhandler_splitdata_paths, names='value')


display(splitdata_chooser, splitdata_idx)
display(splitdata_paths_widget_list[0], splitdata_paths_widget_list[1], splitdata_paths_widget_list[2])

Dropdown(options={'splitdata0009': 0, 'splitdata0010': 1, 'splitdata0011': 2, 'splitdata0012': 3, 'splitdata00…

Output()

Output()

Output()

Output()

# Loading tracking results

In [58]:
#trackingSavePath = '/home/liam/Downloads/fight_tracking_results_FishTank20200130_153857.h5'
trackingSavePath = '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/fight_tracking_results_FishTank20200130_153857.h5'
with h5py.File(trackingSavePath, 'r') as hf:
    tracks_3D = hf['tracks_3D'][:]
    tracks_imCoords = hf['tracks_imCoords'][:]

In [59]:
tracks_3D.shape

(501943, 2, 3, 3)

In [60]:
tracks_imCoords.shape

(3, 501943, 2, 3, 2)

In [97]:
totalnumFrames = tracks_3D.shape[0]

In [118]:
# loading older trajectories
oldPath = '/home/liam/Downloads/FishTank20200130_153857_tracks_3D.h5'
with h5py.File(oldPath, 'r') as hf:
    tracks_3D_old = hf['tracks_3D'][:]
    #tracks_imCoords_old = hf['tracks_imCoords'][:]


In [119]:
tracks_3D_old.shape

(501943, 2, 3, 3)

In [124]:
# load the trajectories from sLEAP tracked in the old way
trackTestPath = '/home/liam/Data/results_from_laetitia/FishTank20200130_153857/old_tracking_on_original_instances_sLEAP.h5'
with h5py.File(trackTestPath, 'r') as hf:
    tracks_3D_test = hf['tracks_3D'][:]
    #tracks_imCoords_old = hf['tracks_imCoords'][:]

In [125]:
tracks_3D_test.shape

(501943, 2, 3, 3)

# Loading Raw network results

In [153]:
# load the old networks WITH tatsuos annotations for training
inst_path = '/home/liam/Data/testing_network_accuracy_in_2_fish/new_original_instances_153857.h5'
with h5py.File(inst_path, 'r') as hf:
    original_instances_old = hf['original_instances'][:]

original_instances_old.shape

(3, 501943, 2, 3, 2)

In [154]:
# load the new networks WITH tatsuos annotations for training, after conversion to original_instances format
inst_path = '/home/liam/Data/testing_network_accuracy_in_2_fish/original_instances_sLEAP.h5'
with h5py.File(inst_path, 'r') as hf:
    original_instances_new = hf['original_instances'][:]

original_instances_new.shape

(3, 501943, 2, 3, 2)

# Quantifying missing data

## In the raw network output

In [156]:
def count_numFish_found(original_instances):
    ''' Return the number of fish found in each frame for each camera view
    '''
    # set all zeros to NaNs
    original_instances = original_instances.astype('float')
    original_instances[original_instances==0] = np.NaN
    numCams, numFrames, numFish, numBps, _ = original_instances.shape
    numFishFound = np.zeros((numCams, numFrames))
    for camIdx in range(numCams):
        for fIdx in range(numFrames):
            fishFound = 0
            for fishIdx in range(numFish):
                if ~np.all(np.isnan(original_instances[camIdx, fIdx, fishIdx])):
                    fishFound += 1
            numFishFound[camIdx, fIdx] = fishFound
    return numFishFound
                    
            
        
def count_numBodypoints_found(original_instances):
    ''' Return the number of bps found in each frame for each camera view
    '''
    # set all zeros to NaNs
    original_instances = original_instances.astype('float')
    original_instances[original_instances==0] = np.NaN
    numCams, numFrames, numFish, numBps, _ = original_instances.shape
    numBps_arr = np.zeros((numCams, numFrames))
    for camIdx in range(numCams):
        for fIdx in range(numFrames):
            numBps_found = 0
            for fishIdx in range(numFish):
                for bpIdx in range(numBps):
                    if ~np.all(np.isnan(original_instances[camIdx, fIdx, fishIdx, bpIdx])):
                        numBps_found += 1
            #import pdb; pdb.set_trace()
            numBps_arr[camIdx, fIdx] = numBps_found
    return numBps_arr



# get the number of fish and bps found by for each data
t0 = time.time()

# old
numFish_found_old = count_numFish_found(original_instances_old)
numFish_found_mean_old = np.mean(numFish_found_old, axis=1)
numBps_found_old = count_numBodypoints_found(original_instances_old)
numBps_found_mean_old = np.mean(numBps_found_old, axis=1)

# sLEAP
numFish_found_new = count_numFish_found(original_instances_new)
numFish_found_mean_new = np.mean(numFish_found_new, axis=1)
numBps_found_new = count_numBodypoints_found(original_instances_new)
numBps_found_mean_new = np.mean(numBps_found_new, axis=1)

tE = time.time()
print(tE-t0)

126.00515294075012


In [157]:
print('-- Old --')
print(numFish_found_mean_old)
print(numBps_found_mean_old)
print()

print('-- new --')
print(numFish_found_mean_new)
print(numBps_found_mean_new)
print()

-- Old --
[1.5443905  1.93911859 1.3730822 ]
[4.47758809 5.70549246 4.08892245]

-- new --
[1.80652983 1.9634082  1.83032934]
[5.37825809 5.86898911 5.43987066]



## At the trajectory level

In [173]:
# How many frames have full information? 

t0 = time.time()

full_data_test = np.zeros((totalnumFrames,))
full_data_old = np.zeros((totalnumFrames,))

for fIdx in range(totalnumFrames):

    f_data_test = tracks_3D_test[fIdx]
    f_data_old = tracks_3D_old[fIdx]

    if ~np.any(np.isnan(f_data_test)):
        full_data_test[fIdx] = 1
        
    if ~np.any(np.isnan(f_data_old)):
        full_data_old[fIdx] = 1
        
        
counts, vals = np.unique(full_data_old, return_counts=True)
old_full_data_summary = list(zip(counts, vals))
old_full_fraction = old_full_data_summary[1][1] / totalnumFrames

counts, vals = np.unique(full_data_test, return_counts=True)
test_full_data_summary = list(zip(counts, vals))
test_full_fraction = test_full_data_summary[1][1] / totalnumFrames

print('-- Percentage of frames with full information --')
print('Old networks: {0} %'.format(old_full_fraction*100))
print('New networks: {0} %'.format(test_full_fraction*100))
     
        
print()
print('----')
tE = time.time()
print('time: ', tE-t0, 's')


-- Percentage of frames with full information --
Old networks: 68.07705257369861 %
New networks: 85.2467312025469 %

----
time:  5.74419379234314 s


In [172]:
# How many frames have at least some information

t0 = time.time()

some_fish_data_test = np.zeros((totalnumFrames,))
some_fish_data_old = np.zeros((totalnumFrames,))

for fIdx in range(totalnumFrames):

    f_data_test = tracks_3D_test[fIdx]
    f_data_old = tracks_3D_old[fIdx]

    if ~np.all(np.isnan(f_data_test)):
        some_fish_data_test[fIdx] = 1
        
    if ~np.all(np.isnan(f_data_old)):
        some_fish_data_old[fIdx] = 1
        
        
counts, vals = np.unique(some_fish_data_old, return_counts=True)
old_some_fish_data_summary = list(zip(counts, vals))
old_some_fish_fraction = old_some_fish_data_summary[1][1] / totalnumFrames

counts, vals = np.unique(some_fish_data_test, return_counts=True)
test_some_fish_data_summary = list(zip(counts, vals))
test_some_fish_fraction = test_some_fish_data_summary[1][1] / totalnumFrames

print('-- Percentage of frames with some information --')
print('Old networks: {0} %'.format(old_some_fish_fraction*100))
print('New networks: {0} %'.format(test_some_fish_fraction*100))
     
        
print()
print('----')
tE = time.time()
print('time: ', tE-t0, 's')

-- Percentage of frames with some information --
Old networks: 87.33401999828666 %
New networks: 97.5983328784344 %

----
time:  5.816539287567139 s


In [ ]:
Full info: 85.2467312025469 %
Some info: 97.59 %

# Plotting

In [174]:
def plot_ts(w=0):
    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(1,1,1)
    
    fishIdx=0
    bpIdx=2
    f1hx = tracks_3D[0+w:100+w, fishIdx, bpIdx, 0] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 0])
    #f1hy = tracks_3D[0+w:100+w, fishIdx, bpIdx, 1] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 1])
    #f1hz = tracks_3D[0+w:100+w, fishIdx, bpIdx, 2] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 2])
    
    fishIdx=0
    bpIdx=2
    ## USING SAME MEAN FOR SUBTRACTION
    f1hx_old = tracks_3D_old[0+w:100+w, fishIdx, bpIdx, 0] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 0])
    #f1hy = tracks_3D[0+w:100+w, fishIdx, bpIdx, 1] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 1])
    #f1hz = tracks_3D[0+w:100+w, fishIdx, bpIdx, 2] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 2])
    
    
    fishIdx=0
    bpIdx=2
    ## USING SAME MEAN FOR SUBTRACTION
    f1hx_test = tracks_3D_test[0+w:100+w, fishIdx, bpIdx, 0] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 0])
    #f1hy = tracks_3D[0+w:100+w, fishIdx, bpIdx, 1] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 1])
    #f1hz = tracks_3D[0+w:100+w, fishIdx, bpIdx, 2] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 2])
    
    
    ax.plot(f1hx, label='f1hx_new')
    ax.plot(f1hx_old, label='f1hx_old')
    ax.plot(f1hx_test, label='f1hx_test')
    
#     ax.plot(f1hx, label='f1hx')
#     ax.plot(f1hy, label='f1hy')
#     ax.plot(f1hz, label='f1hz')
    ax.set_ylim(-10,10)
    fig.canvas.draw()
    ax.legend()
    
interact(plot_ts, w=widgets.IntSlider(min=0,max=totalnumFrames,step=1,value=10))

interactive(children=(IntSlider(value=10, description='w', max=501943), Output()), _dom_classes=('widget-inter…

<function __main__.plot_ts(w=0)>

# Analysis

In [10]:
global_fstart = 80000
global_fstop = global_fstart + 200

In [11]:
# get the local info
splitdata_idx1, local_fstart = splitman.return_splitdata_folder_and_local_idx_for_global_frameIdx(global_fstart, return_splitdataIdx=True)
splitdata_name1 = splitdata_names[splitdata_idx1]
print(splitdata_idx1)
print(splitdata_name1)
print(local_fstart)
print()
splitdata_idx2, local_fstop = splitman.return_splitdata_folder_and_local_idx_for_global_frameIdx(global_fstop, return_splitdataIdx=True)
splitdata_name2 = splitdata_names[splitdata_idx2]
print(splitdata_idx2)
print(splitdata_name2)
print(local_fstop)

13
splitdata0022
2000

13
splitdata0022
2200


In [12]:
# get the movie paths
# idx1 == idx2
paths = splitman.splitdata_paths[splitdata_idx1]
video_paths = [path +'.mp4' for path in paths]
video_paths

['/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/D_xz/splitdata0022.mp4',
 '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/E_xy/splitdata0022.mp4',
 '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/F_yz/splitdata0022.mp4']

In [89]:
# gather the data for this section of frames
xz_tracks = np.copy(tracks_imCoords[0, global_fstart:global_fstop])
xy_tracks = np.copy(tracks_imCoords[1, global_fstart:global_fstop])
yz_tracks = np.copy(tracks_imCoords[2, global_fstart:global_fstop])
tra_3D = np.copy(tracks_3D[global_fstart:global_fstop])
numFrames = global_fstop - global_fstart

numFrames

200

In [57]:
def update(w=1):
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(1,1,1)
    frame = make_3D_plot_of_frame_positions(tra_3D[w])
    ax.imshow(frame)
    fig.canvas.draw()
    
interact(update, w=widgets.IntSlider(min=0,max=numFrames,step=1,value=0))

interactive(children=(IntSlider(value=0, description='w', max=200), Output()), _dom_classes=('widget-interact'…

<function __main__.update(w=1)>

In [68]:
xz_tracks.shape

(200, 2, 3, 2)

In [90]:
# interpolate the centers

from scipy.interpolate import interp1d
import numpy as np
from scipy import interpolate

def fill_nan(A):
    '''
    Thanks - https://stackoverflow.com/a/9815522
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

# interpolate the fish_pecs so we dont blink
xz_heads = np.copy(xz_tracks[:,:,0])
xy_heads = np.copy(xy_tracks[:,:,0])
yz_heads = np.copy(yz_tracks[:,:,0])
xz_heads_interpd = np.copy(xz_heads)
xy_heads_interpd = np.copy(xy_heads)
yz_heads_interpd = np.copy(yz_heads)

for fishIdx in range(2):
    xz_heads_x = np.copy(xz_heads[:, fishIdx, 0])
    xz_heads_y = np.copy(xz_heads[:, fishIdx, 1])
    xz_heads_x_interpd = fill_nan(xz_heads_x)
    xz_heads_y_interpd = fill_nan(xz_heads_y)
    xz_heads_interpd[:, fishIdx, 0] = xz_heads_x_interpd
    xz_heads_interpd[:, fishIdx, 1] = xz_heads_y_interpd
    
    xy_heads_x = np.copy(xy_heads[:, fishIdx, 0])
    xy_heads_y = np.copy(xy_heads[:, fishIdx, 1])
    xy_heads_x_interpd = fill_nan(xy_heads_x)
    xy_heads_y_interpd = fill_nan(xy_heads_y)
    xy_heads_interpd[:, fishIdx, 0] = xy_heads_x_interpd
    xy_heads_interpd[:, fishIdx, 1] = xy_heads_y_interpd
    
    yz_heads_x = np.copy(yz_heads[:, fishIdx, 0])
    yz_heads_y = np.copy(yz_heads[:, fishIdx, 1])
    yz_heads_x_interpd = fill_nan(yz_heads_x)
    yz_heads_y_interpd = fill_nan(yz_heads_y)
    yz_heads_interpd[:, fishIdx, 0] = yz_heads_x_interpd
    yz_heads_interpd[:, fishIdx, 1] = yz_heads_y_interpd
    
    
crop_centers = [xz_heads_interpd, xy_heads_interpd, yz_heads_interpd]


In [91]:
movieSavePath = '/home/liam/temp/mov1.mp4'

make_3panel_movie(video_paths, xy_tracks, xz_tracks, yz_tracks, movieSavePath, 
                  crop_centers, cropSize=(80,80), bw=20, padding_depth=70,
                  fstart=local_fstart, fstop=local_fstop)

fstart 2000
fstop 2200
2000
2100


# Making a movie of results

In [75]:
def _pad_frame(frame, padding_depth):
    ''' Return a padded version of the frame '''
    # grab the padding from the image
    padding = frame[0:padding_depth,:]
    # make the padded frame
    frame = np.vstack((padding, frame, padding))
    return frame


In [82]:
# Make a movie

def make_3panel_movie(video_paths, xy_tracks, xz_tracks, yz_tracks, movieSavePath, 
                      crop_centers, cropSize=(80,80), bw=20, padding_depth=70,
                      fstart=0, fstop=6000):
    """ Visualize the sLEAP and idTracker data drawn onto the movie
    
    -- args --
    video_path: the filepath of the splitdata.mp4
    **_tracks: the tracked trajectories from the ** camera
    movieSavePath: the filepath to save the new mp4 to
    crop_centers: these are interpolated timeseries of head position, that are
                  used from cropping when making the movie
    
    """
    print('fstart', fstart)
    print('fstop', fstop)
    
    # set up the video maker
    fps = 100
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    resolution = (480+bw+bw, 160) # cols/rows
    out = cv2.VideoWriter(movieSavePath, fourcc, fps, resolution)
    
    # parse some shapes
    numFrames, numFish, numBodyPoints, _ = xy_tracks.shape
    if numFrames > fstop:
        fstop = numFrames
    
    # make a list of colors
    colors = [(0,255,0), (255,255,255)] # blue and green
    
    # open the caps
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    
    # pack up the tracks
    tracks = [xz_tracks, xy_tracks, yz_tracks]
    
    frame_index = 0
    while caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():
            
        # grab the 3 images
        ret_xz, frame_xz = caps[0].read()
        if not ret_xz:
            break
        ret_xy, frame_xy = caps[1].read()
        if not ret_xy:
            break
        ret_yz, frame_yz = caps[2].read()
        if not ret_yz:
            break
        frames = [frame_xz, frame_xy, frame_yz]
        
        if frame_index < fstart:
            frame_index += 1
            continue
    
        # update message
        if np.mod(frame_index, 100) == 0:
            print(frame_index)
        
        # draw the info
        for camIdx in range(3):
            frame = frames[camIdx]
            trks = tracks[camIdx]
            for fishIdx in range(numFish):
                for bpIdx in range(numBodyPoints):
                    # get the data
                    imCoord = trks[frame_index-fstart, fishIdx, bpIdx]
                    #get the color
                    color = colors[fishIdx]
                    color = ( int (color[ 0 ]), int(color[ 1 ]), int(color[ 2 ]))
                    if np.all(np.isnan(imCoord)):
                        continue
                    cv2.circle(frame, (int(imCoord[0]), int(imCoord[1])), radius=2, color=color, thickness=-1)
                    
        # take crops around fish1
        crops = []
        for camIdx in range(3):
            fish1_head = crop_centers[camIdx][frame_index-fstart, 0]
            # get the center of the crop, adjusted for padding
            cropCenter = [fish1_head[0], fish1_head[1]+padding_depth]
            
            if np.all(np.isnan(fish1_head)):
                crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
                crops.append(crop)
            else:
                frame = frames[camIdx]
                frame = _pad_frame(frame, padding_depth)
                crop = frame[int(cropCenter[1]-cropSize[1]):int(cropCenter[1]+cropSize[1]),
                             int(cropCenter[0]-cropSize[0]):int(cropCenter[0]+cropSize[0])]
                crops.append(crop)
            
        # combine the crops
        barrier = np.zeros((cropSize[0]*2, bw, 3), dtype=np.uint8)
        movFrame = np.hstack([crops[0], barrier, crops[1], barrier, crops[2]])
        #movFrame = cv2.resize(movFrame, resolution, interpolation = cv2.INTER_AREA)
        
        # write the frame number
        x1, y1 = 10,10
        label = str(frame_index).zfill(6)
        cv2.putText(movFrame,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)
        # write the frame
        out.write(movFrame)
        frame_index += 1
        if frame_index >= fstop:
            break
        
    # close everything before we finish
    for capIdx in range(3):
        caps[capIdx].release()
    out.release() 
    return

In [53]:
# This is a function for making a 3D visualization of the fish in 3D for a frame
# This function is useful because it returns a regular image array, not a matplotlib figure or something
# This makes it much easier to write in a movie

def make_3D_plot_of_frame_positions(f_positions):
    ''' Given the frame positions, return a 3D plot of the positions of this fish in
        array form
    '''
    # turn off interactive plots for now
    plt.ioff()

    # get the number of bps and numFish
    numFish = f_positions.shape[0]
    numBps = f_positions.shape[1]

    # make a figure and canvas
    fig = plt.figure(figsize=(15,15))
    fig.tight_layout(pad=0)
    canvas = FigureCanvas(fig)
    fig.add_subplot(111, projection='3d')
    ax = fig.gca()

    #loop over fish
    for f in range(numFish):

            
        # decide on the color
        if f == 0:
            c_real = [0,0,1,1]
            c_projection = [0,0,1,0.15]
        elif f == 1:
            c_real = [1,0,0,1]
            c_projection = [1,0,0,0.15]
        else:
            raise TypeError('Need to define more colors')

        # get the positions of the bps for this fish
        f_pos = f_positions[f]

        #  -- real -- #
        # plot each of the points
        ax.scatter(f_pos[0,0], f_pos[0,1], f_pos[0,2], c=c_real, marker='D', depthshade=False, s=40)
        ax.scatter(f_pos[1,0], f_pos[1,1], f_pos[1,2], c=c_real, marker='x', depthshade=False, s=40)
        ax.scatter(f_pos[2,0], f_pos[2,1], f_pos[2,2], c=c_real, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(f_pos[:, 0], f_pos[:, 1], f_pos[:, 2], color=c_real, linewidth=2)

        # -- XY projection -- #
        # we want to set the Z coord of everypoint to 0
        # color is red with alpha layer
        ax.scatter(f_pos[0,0], f_pos[0,1], 0, c=c_projection, marker='D', depthshade=False, s=40)
        ax.scatter(f_pos[1,0], f_pos[1,1], 0, c=c_projection, marker='x', depthshade=False, s=40)
        ax.scatter(f_pos[2,0], f_pos[2,1], 0, c=c_projection, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(f_pos[:, 0], f_pos[:, 1], np.zeros((numBps,)), c=c_projection, linewidth=2)

        # -- XZ projection -- #
        # we want to set the Y coord of everypoint to 0
        # color is red with alpha layer
        ax.scatter(f_pos[0,0], 0, f_pos[0,2], c=c_projection, marker='D', depthshade=False, s=40)
        ax.scatter(f_pos[1,0], 0, f_pos[1,2], c=c_projection, marker='x', depthshade=False, s=40)
        ax.scatter(f_pos[2,0], 0, f_pos[2,2], c=c_projection, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(f_pos[:,0], np.zeros((numBps,)), f_pos[:,2], c=c_projection, linewidth=2)

        # -- YZ projection -- #
        # we want to set the X coord of everypoint to 0
        # color is red with alpha layer
        ax.scatter(0, f_pos[0,1], f_pos[0,2], c=c_projection, marker='D', depthshade=False, s=40)
        ax.scatter(0, f_pos[1,1], f_pos[1,2], c=c_projection, marker='x', depthshade=False, s=40)
        ax.scatter(0, f_pos[2,1], f_pos[2,2], c=c_projection, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(np.zeros((numBps,)), f_pos[:,1], f_pos[:,2], c=c_projection, linewidth=2)


    # make the plot pretty
    ax.set_xlabel('X (cm)', fontsize=20)
    ax.set_ylabel('Y (cm)', fontsize=20)
    ax.set_zlabel('Z (cm)', fontsize=20)
    ax.set_xlim(0,40)
    ax.set_ylim(0,40)
    ax.set_zlim(0,40)
    ticks = [0, 10, 20, 30, 40]
    ax.set_xticks(ticks, minor=False)
    ax.set_yticks(ticks, minor=False)
    #ax.grid(False)

    # set the angle of viewing
    elevation_angle = 10
    azimuthal_angle = 45
    ax.view_init(elevation_angle, azimuthal_angle)

    # draw the canvas
    canvas.draw()

    # get the width and height of the figure, and turn them to type ints
    height, width = fig.get_size_inches() * fig.get_dpi()
    width = width.astype('int')
    height = height.astype('int')

    # convert the figure to an array of the correct size
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(width, height, 3)

    # close the figure - we are done. You can also turn back on interactive plots now
    plt.close()
    plt.ion()

    # lastly remove some of the white space from around the frame
    # NOTE: IF YOU CHANGE FIGURE SIZE YOU WILL HAVE TO CHANGE THIS
    #       THIS IS A MANUAL WAY TO REMOVE WHITESPACE
    #img = img[325:1225, 250:1265]
    img = img[300:900, 150:1000]

    # return the image
    return img

# Displaying a movie

In [31]:
camIdx = 0
movPath = splitdata_paths_widget_list[camIdx].outputs[0]['data']['text/plain'][1:-1] + '.mp4'

In [36]:
video = Video.from_file(movPath)

In [37]:

video

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\xf95\xf1mdat\x00\x…